In [10]:
import json
from queue import Queue
import os
from bs4 import BeautifulSoup
import re

In [2]:
with open('../Topic/TopicCooccurrency-mt3.json','r') as f:
    topicCooccurrency = json.load(f)
topics = [topic for topic in topicCooccurrency.keys() if len(topicCooccurrency[topic])>1]
with open('topics.json','w') as f:  # 保存涉及topics
    json.dump(topics,f,indent=1)

In [8]:
def compactTopic(topic):
    charset = set(['+', '#'])
    result = filter(lambda c: c.isdigit() or c.isalpha() or c in charset, topic)
    return("".join(result)) # 1234
def occurTime(topic):
    return topicCooccurrency[topic][topic]


In [9]:
compactTopic('Direcci\u00f3n General @prende.mx')

'DirecciónGeneralprendemx'

In [5]:
topiccompactdict = {}
for topic in topics:
    ex = compactTopic(topic)
    if ex not in topiccompactdict:
        topiccompactdict[ex] = topic
    elif occurTime(topiccompactdict[ex]) < occurTime(topic):
        topiccompactdict[ex] = topic
with open('../Topic/Topic-synonyms.json','r') as f:
    topicSynonyms = json.load(f)
for topic, syn in topicSynonyms.items():
    if not topic in topicCooccurrency:
        continue
    for s in syn:
        sc = compactTopic(s)
        if sc not in topiccompactdict:
            topiccompactdict[sc] = topic
        elif occurTime(topiccompactdict[sc]) < occurTime(topic):
            topiccompactdict[sc] = topic
with open('topiccompactdict.json','w') as f:  # 保存涉及topics
    json.dump(topiccompactdict,f,indent=1)

In [6]:
with open('Pageleaf-redirect-encode.json','r') as f:
    PageleafRedirectEncode = json.load(f)

In [12]:
Pagematched = {}
for id, page in PageleafRedirectEncode.items():
    pagecompact = page
    if page.find('(') != -1:
        # remove '(' ')' and content in it
        pagecompact = re.sub(r"\(.*?\)", "", page)
    pagecompact = pagecompact.lower()
    pagecompact = compactTopic(pagecompact)
    if pagecompact in topiccompactdict:
        Pagematched[id] = {'Wikipage':page, 'topic':topiccompactdict[pagecompact]}
with open('Pagematched.json','w') as f:  # 保存涉及topics
    json.dump(Pagematched,f,indent=1)

In [7]:
charset = set()
extras = set()
for topic in topics:
    result = filter(lambda c: not c.isdigit() and not c.isalpha() and not c in charset, topic)
    for c in result:
        extras.add(c)
print(extras)

{'+', '-', '.', '#'}


In [15]:
deletetype = ['video game', 'color', 'film', 'comics', 'TV series', 'franchise', 'book', 'character', 'magazine', 'song', 'novel', 'album', 'journal']
newPagematched = {}
for page, info in Pagematched.items():
    if info['Wikipage'].find('(') != -1:
        typ = info['Wikipage'][info['Wikipage'].find('(')+1:info['Wikipage'].find(')')].lower()
        td = 0
        for d in deletetype:
            if d in typ:
                td = 1
                break
        if td == 1:
            continue
    newPagematched[page] = info
with open('Pagematched.json','w') as f:
    json.dump(newPagematched,f,indent=1)

In [20]:
with open('Pagematched.json','r') as f:
    Pagematched = json.load(f)

In [18]:
toccur = set()
Pagematched = {}
for page, info in newPagematched.items():
    if info['Wikipage'].find('(') != -1 and info['topic'] in toccur:
        continue
    Pagematched[page] = info
    toccur.add(info['topic'])
with open('Pagematched.json','w') as f:
    json.dump(Pagematched,f,indent=1)

In [19]:
with open('Pagematched.json','w') as f:
    json.dump(Pagematched,f,indent=1)